In [1]:
import os
os.environ["PATH"] += os.pathsep + "/opt/conda/envs/team05/bin"
os.environ["PATH"] += os.pathsep + "/workspace/chimera/bin"
from glob import glob
import pandas as pd
import numpy as np
from Bio import PDB
import os
from rdkit.Chem.rdmolfiles import MolFromSmiles, MolToSmiles
from Bio.PDB import Select, PDBIO
from Bio.PDB.PDBParser import PDBParser
from biopandas.mol2 import PandasMol2

In [2]:
pdb_path = os.path.abspath("datasets/scPDB/")
info_path = os.path.abspath("datasets/")

complex_list = [name for name in os.listdir(pdb_path) if not name.startswith(".")]

In [ ]:
%%bash -s $pdb_path
export PATH="/opt/conda/envs/team05/bin:/workspace/chimera/bin:$PATH"

path=$1

for file in $path/*/protein.mol2; do
    output_path=${file%.mol2}.pdb
    echo -e "open $file \n write format pdb 0 $output_path \n stop" | chimera --nogui
done

> Opening /workspace/datasets/scPDB/10mh_1/protein.mol2...
Opening VRML model in 10mh_SAH_1_protein - Nucleotides...
Model 0 (10mh_SAH_1_protein) appears to be a protein without secondary structure assignments.
Automatically computing assignments using 'ksdssp' and parameter values:
  energy cutoff -0.5
  minimum helix length 3
  minimum strand length 3
Use command 'help ksdssp' for more information.

Computing secondary structure assignments...
Computed secondary structure assignments (see reply log)
10mh_SAH_1_protein opened
Opening VRML model in 10mh_SAH_1_protein - Nucleotides...
> Wrote /workspace/datasets/scPDB/10mh_1/protein.pdb
> > Opening /workspace/datasets/scPDB/11bg_2/protein.mol2...
Model 0 (11bg_U2G_2_protein) appears to be a protein without secondary structure assignments.
Automatically computing assignments using 'ksdssp' and parameter values:
  energy cutoff -0.5
  minimum helix length 3
  minimum strand length 3
Use command 'help ksdssp' for more information.

Computi

In [ ]:
def remove_HEATM_PDBbind(input_list, path):

    class NonHetSelect(Select):
        def accept_residue(self, residue):
            return 1 if residue.id[0] == " " else 0
    
    for pdb in input_list:

        src_file = f"{pdb_path}/{pdb}/protein.pdb"
        des_file = f"{pdb_path}/{pdb}/remove_HEATM_protein.pdb"
        
        pdb = PDBParser().get_structure(pdb, src_file)
        io = PDBIO()
        io.set_structure(pdb)
        io.save(des_file, NonHetSelect()) 

In [ ]:
remove_HEATM_PDBbind(complex_list, pdb_path)

In [ ]:
from scipy.spatial import distance_matrix
from multiprocessing import Process, Queue, Pool

In [ ]:
pdb_parser = PDB.PDBParser(QUIET=True)

In [ ]:
amino_acids_short = {"ALA":"A", "ARG":"R", "ASN":"N", "ASP":"D", "CYS":"C", "GLU":"E", "GLN":"Q", "GLY":"G", "HIS":"H", "ILE":"I", "LEU":"L", "LYS":"K", "MET":"M", "PHE":"F", "PRO":"P", "SER":"S", "THR":"T", "TRP":"W", "TYR":"Y", "VAL":"V", "SEC":"U", "PYL":"O"}

In [ ]:
def parallelize_dataframe(df, func, num_partitions=10):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_partitions)
    results = pool.map(func, df_split)
    pool.close()
    pool.join()
    return results

In [ ]:
def get_protein(scpdb):
    try:
        """ Load protein info """
        # remove_HEATM_protein.pdb 파일을 불러옴
        structure = pdb_parser.get_structure(scpdb, f"{pdb_path}/{scpdb}/remove_HEATM_protein.pdb")
        # 초기화: 결과 저장용 리스트 및 인덱스
        chain_name_list, pdb_sequence_list, seq_lengths_list, protein_atom_coords, protein_residue_list, reindex = list(), list(), list(), list(), list(), 0

        for chain_name in list(structure[0].child_dict.keys()):
            chain = structure[0][chain_name]

            residue_number, pdb_sequence = list(), ""
            for residue in chain.get_residues():
                # 아미노산 이름이 3-letter → 1-letter 딕셔너리에 있는 경우만 처리
                if residue.resname in amino_acids_short.keys():
                    residue_number.append(reindex)
                    pdb_sequence += amino_acids_short[residue.resname]

                    for atom in residue:
                        # 원자 좌표 저장
                        protein_atom_coords.append(atom.get_coord())
                        # 어떤 residue에 속하는지 저장
                        protein_residue_list.append(reindex)
                    reindex += 1

            if len(pdb_sequence) != 0:
                chain_name_list.append(chain_name)
                pdb_sequence_list.append(pdb_sequence)
                seq_lengths_list.append(len(pdb_sequence)) 
                #residue_number_list.append(",".join(map(str, residue_number)))

        """ Load pocket info """
        bi_df = PandasMol2().read_mol2(f"{pdb_path}/{scpdb}/site.mol2").df
        bi_df = bi_df.drop_duplicates("subst_name")
        bi_df = bi_df.loc[bi_df.subst_name.map(lambda a: a[:3] in amino_acids_short.keys())]
        bi_xyz_coordi = bi_df.iloc[:, [2,3,4]].values
        bi_x, bi_y, bi_z = bi_xyz_coordi[:, 0], bi_xyz_coordi[:, 1], bi_xyz_coordi[:, 2] 

        protein_atom_coords, binding_index = np.array(protein_atom_coords), list()

        for i, j, k in zip(bi_x, bi_y, bi_z):

            tmp_coordi = np.array([i, j, k], dtype = np.float32)

            # 정확히 일치하는 좌표를 단백질 원자 좌표에서 찾음
            ind = np.where((protein_atom_coords == tmp_coordi).all(axis = 1))[0][0]
            binding_index.append(protein_residue_list[ind])

        binding_index = list(map(str, binding_index))

        total_seq_lengths = np.sum(np.array(seq_lengths_list))
        seq_lengths_list = list(map(str, seq_lengths_list))        
        
        return ",".join(chain_name_list), ",".join(pdb_sequence_list), total_seq_lengths, ",".join(seq_lengths_list), ",".join(binding_index)
    
    except Exception as e:
        print(scpdb, e)
        return None

In [ ]:
def get_raw_protein_info_bulk(df):
    return df.scPDB.map(get_protein)

In [ ]:
pdb = [scpdb[:-2] for scpdb in complex_list]

In [ ]:
data_df = pd.DataFrame({"scPDB":complex_list, "PDB":pdb})
data_df

In [ ]:
info_results = parallelize_dataframe(data_df, get_raw_protein_info_bulk, num_partitions = 5)

In [ ]:
info_results = pd.concat(info_results)

In [ ]:
data_df["Chain"] = info_results.map(lambda a: a[0] if a is not None else None)

In [ ]:
data_df["Sequence"] = info_results.map(lambda a: a[1] if a is not None else None)

In [ ]:
data_df["Total_seq_lengths"] = info_results.map(lambda a: a[2] if a is not None else None)

In [ ]:
data_df["Chain_seq_lengths"] = info_results.map(lambda a: a[3] if a is not None else None)

In [ ]:
data_df["BS"] = info_results.map(lambda a: a[4] if a is not None else None)

In [ ]:
data_df = data_df.loc[data_df.Sequence.isna()==False].reset_index(drop=True)
data_df = data_df.loc[data_df.Chain != " "].reset_index(drop=True)

data_df

In [ ]:
lengths = data_df.Total_seq_lengths.values

data_df = data_df[lengths <= 1500].reset_index(drop=True)
data_df

In [ ]:
from rdkit.Chem.rdmolfiles import MolFromSmiles, MolToSmiles
def read_file(file):
    return file.readlines()

def add_ligand(scpdb):

    mol = f"{pdb_path}/{scpdb}/ligand.mol2"

    #command = f"obabel -imol2 {mol} -osmi -O tmp.smi"
    command = f"obabel -imol2 {mol} -osmi -xC | obabel -ismi -osmi -xk -O tmp.smi"
    os.system(command)

    smiles = read_file(open("tmp.smi"))[0].split('\t')[0].strip()
    
    try:
        smiles = MolToSmiles(MolFromSmiles(smiles),isomericSmiles = False, kekuleSmiles = True)
        return smiles
    
    except Exception as e:
        print(scpdb, e)
        return None 

In [ ]:
SMILES = data_df.scPDB.map(add_ligand)

In [ ]:
data_df["SMILES"] = SMILES

In [ ]:
data_df = data_df.loc[data_df.SMILES.isna()==False].reset_index(drop=True)
data_df

In [ ]:
def get_SMILES_length(df):
    index = [True if len(smi) <= 160 else False for smi in df.SMILES.values]
    return index

In [ ]:
smiles_index = get_SMILES_length(data_df)

In [ ]:
data_df = data_df.loc[smiles_index].reset_index(drop=True)
data_df

In [ ]:
data_df = data_df.iloc[:, [0, 3, 6]]

In [ ]:
data_df["scPDB"] = data_df["scPDB"].apply(lambda x: x[:-2] if "_" in x and x[-2] == "_" else x)
data_df.to_csv("datasets/scPDB_data.tsv", sep = "\t", index = False)